# Overview
This notebook is used to create a regression model that predicts the median house price within a block based on a number of features. The dataset used is the "California Housing Prices" dataset which is available at https://www.kaggle.com/datasets/camnugent/california-housing-prices/data. Here is a summary of the Kaggle description of the dataset:

The dataset discussed is initially featured in the paper "Sparse Spatial Autoregressions" by R. Kelley Pace and Ronald Barry, published in the Statistics & Probability Letters, 33 (1997) 291-297. The author of the Kaggle project encountered this dataset in the second chapter of Aurélien Géron's book, "Hands-On Machine Learning with Scikit-Learn and TensorFlow." This dataset was a modified version of the California Housing dataset available at Luís Torgo's page (University of Porto) http://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html.
    
This dataset contains information about houses in California districts derived from the 1990 California census. While it may not be suitable for predicting current housing prices, it serves as a great introduction for building machine learning models due to its need for basic data cleaning, a clear and easily comprehensible list of features, and an optimal size that strikes a balance between being overly simplistic and excessively complex.

The dataset contains 20640 rows and 10 columns. The columns are as follows:
1. longitude (float): A measure of how far west a house is; a higher value is farther west
2. latitude (float): A measure of how far north a house is; a higher value is farther north
3. housing_median_age (float): Median age of houses within a block; a lower number is a newer building
4. total_rooms (float): Total number of rooms within a block
5. total_bedrooms (float): Total number of bedrooms within a block
6. population (float): Total number of people residing within a block
7. households (float): Total number of households, a group of people residing within a home unit, for a block
8. median_income (float): Median income for households within a block of houses (measured in tens of thousands of US Dollars)
9. median_house_value (float, target): Median house value for households within a block (measured in US Dollars)
10. ocean_proximity (string): Location of the house w.r.t. ocean/sea

This notebook performs the following steps to create a regression model:
1. Data Exploration
2. Data Preprocessing
3. Model Training and Evaluation

The creation of this notebook has taken inspiration from the following sources:
- Aurélien Géron's "02_end_to_end_machine_learning_project.ipynb" python notebook in "handson-ml2" git repository. https://github.com/ageron/handson-ml2/blob/master/02_end_to_end_machine_learning_project.ipynb
- Andrey Shtrauss's "Bayesian Regression | House Price Prediction" python notebook in Kaggle. https://www.kaggle.com/code/shtrausslearning/bayesian-regression-house-price-prediction#5-|-FEATURE-ENGINEERING



# Install Python Packages/Libraries

Install the specified Python packages. Here's a breakdown of each package:

`joblib`: A set of tools for pipelining Python jobs. It provides utilities for saving and loading Python objects that make it possible to save scikit-learn models in a format that can be used in production.

`matplotlib`: A plotting library for creating visualizations in Python. It is often used in conjunction with other libraries for data analysis and machine learning.

`numpy`: A fundamental package for scientific computing with Python. It provides support for large, multi-dimensional arrays and matrices, along with mathematical functions to operate on these arrays.

`pandas`: A powerful data manipulation and analysis library. It provides data structures for efficiently storing and manipulating large datasets.

`seaborn`: A statistical data visualization library based on Matplotlib. It provides a high-level interface for creating informative and attractive statistical graphics.

`scikit-learn`: A machine learning library that provides simple and efficient tools for data analysis and modeling. It includes various algorithms for classification, regression, clustering, and more.


In [ ]:
%pip install joblib==1.3.2 matplotlib==3.7.1 numpy==1.23.5 pandas==1.5.3 plotly==5.15.0 scikit-learn==1.2.2 seaborn==0.13.1 "nbformat>=4.2.0"

# Import Packages/Libraries

In addition to the packages/libraries installed above, we will also imported

`typing`: A module that provides support for type hints. Type hints allow you to specify the type of a variable, function parameter, or return value. This helps improve the readability of your code and allows you to catch errors early.

In [ ]:
from typing import List, Tuple
import zipfile
import os
import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sns
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer


# Data Exploration

We want to explore the data to get a better understanding of the dataset. We will use the pandas library to load the dataset into a pandas DataFrame. A DataFrame is a two-dimensional data structure that can store data of different types (including characters, integers, floating point values, categorical data, and more) in columns. It is similar to a spreadsheet or an SQL table. The DataFrame object also has powerful built-in methods for exploring and manipulating these data sets. We will first take a look at the structure of the dataset. Then, we want to explore the data to see if there are any missing values and to see if there are any skewness and outliers. We also want to see if there are any correlations between the features and the target variable. We will use the pandas library to load the dataset and the matplotlib, seaborn, and plotly library to plot the data.

Load the `housing.csv` file into a Pandas DataFrame using the `read_csv()` function. The `read_csv()` function takes in the path to the CSV file as a parameter and returns a DataFrame.

In [ ]:
!unzip dataset.zip -d dataset # Unzip the file if needed.

In [ ]:
csv_path =  # TODO: create variable for dataset path
df =  # TODO: read dataset into dataframe

Display the first 5 rows of the dataframe using `head()` method of the dataframe.

We can see that the data contains 10 columns where the first 9 columns are numerical (continuous) values and the last column is a string (categorical) value. The target variable is the `median_house_value` column.

In [ ]:
# TODO: show the first 5 rows of dataframe


Obtain a concise summary of the dataframe. The `info()` method provides information about the dataframe, including the index range, the data types of each column, the number of non-null values, and memory usage.

In [ ]:
 # TODO: show the info of dataframe

Retrieve the columns of the dataframe using the `columns` attribute of the dataframe.

In [ ]:
 # TODO: show the columns of dataframe

Retrieve the dimensions (rows, columns) of the dataframe using the `shape` attribute of the dataframe.

In [ ]:
# TODO: retrieve the number of rows and columns


Check and handles missing values in the Pandas DataFrame using the `isnull()` method of the dataframe. The `isnull()` method returns a boolean dataframe where each cell is either True or False depending on whether the corresponding cell in the dataframe is null or not. We can then use the `sum()` method to count the number of null values in each column.

In [ ]:
# TODO: check the number of missing values in each column
print('Missing values in each column:\n', , sep='')
print('Original data shape: ', df.shape)
print('='*25)

# TODO: print the rows with missing values and store the row indexes
null_rows =
null_rows_index =
print('Rows with missing values:\n')
null_rows


We have a few options to handle missing values:
- Option 1: We can drop the rows or columns that contain missing values using the `dropna()` method of the dataframe. The `dropna()` method takes in the axis to drop as a parameter. We can use the `axis` parameter to specify whether to drop rows or columns. By default, `axis=0` which drops the rows.
- Option 2: We can replace the missing values with the mean, median, or mode of the column. We will replace the missing values with the median of the column. We can use the `fillna()` method of the dataframe to replace the missing values with the median of the column. The `fillna()` method takes in the value to replace the missing values with as a parameter. We can use the `median()` method of the dataframe to obtain the median of the column.
- Option 3: We can replace the missing values with KNN imputation. KNN imputation is a method that uses the k-nearest neighbors algorithm to impute missing values. It works by finding the k-nearest neighbors of data points with missing values and then imputing the missing values with the mean of the neighbors. We will use the `KNNImputer()` from the `sklearn.impute` module to impute the missing values. The `KNNImputer()` takes in the number of neighbors and weights to use as a parameter. We will use 10 neighbors and weight points by the inverse of their distance. We can then use the `fit_transform()` method of the `KNNImputer()` object to impute the missing values. The `fit_transform()` method takes in the dataframe as a parameter and returns a numpy array. We can then convert the numpy array to a dataframe using the `DataFrame()` from the `pandas` library. Reference link for `KNNImputer()`: https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html

In [ ]:
# TODO: Option 1: drop the rows with missing values
df_dropped_na =
print('Missing values after dropped na:\n', df_dropped_na.isnull().sum(), sep='')
print('Data shape after dropping nan:', df_dropped_na.shape)
print('='*25)

In [ ]:
# TODO: Option 2: fill in the missing values with the median of the column
df_median_filled =
print('Missing values after filling with median values:\n', df_median_filled.isnull().sum(), sep='')
print('Data shape after filling with median values:', df_median_filled.shape)

# check the null rows again
print('Median values for each column:\n', df.median(), sep='')
df_median_filled.iloc[null_rows_index]

In [ ]:
# Option 3: use K-nearest neighbor imputation to fill in the missing values
# see https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html
# for more details and parameters on the KNNImputer
imputer =  # TODO: create a KNNImputer object

# TODO: separate the categorical and numerical columns as
# KNNImputer can only handle numerical columns
cat_cols =
num_cols =

# TODO: impute the missing values in the numerical columns
# the imputer returns a numpy array with imputed values
df_num_filled =

# TODO: convert the imputed numpy array back to a pandas dataframe
df_num_filled =

# TODO: concatenate the imputed numerical columns with the categorical columns
df_filled =
print('Missing values after KNN impute:\n', df_filled.isnull().sum(), sep='')
print('Data shape after KNN impute:', df_filled.shape)

# check the null rows again
df_filled.iloc[null_rows_index]

Check the number of unique values in each column of dataframe using the `nunique()` method of the dataframe.

In [ ]:
# Check the number of unique values in each column
df.nunique()

From the `df.info()` and `df.head(5)` cell, we know that `ocean_proximity` is an object/string data type. We can use the `value_counts()` method to count the occurrences of each unique value in the column named `ocean_proximity`.

In [ ]:
# Check the occurences of each unique value in the 'ocean_proximity' column
df['ocean_proximity'].value_counts()

We can also display a pie chart to visualize the distribution of values in the `ocean_proximity` column using the `plot.pie()` method of the Pandas Series.

In [ ]:
# Create a plt figure. Set the figure size and background color to white in case your editor GUI uses a dark theme
plt.figure(figsize=(5, 5), facecolor='white')

# Show the percentage of each category using a pie chart
# plot.pie() plots a pie chart on the figure attached to the current cell
# autopct='%1.2f%%' shows the percentage of each category with 2 decimal places
df['ocean_proximity'].value_counts().plot.pie(autopct='%1.2f%%')

plt.show()

Generate descriptive statistics of the numerical columns in the dataframe using the `describe()` method of the dataframe. The `describe()` method computes summary statistics of numerical columns, including count, mean, standard deviation, minimum, maximum, and the 25, 50, and 75 percentiles.

In [ ]:
# Get the summary statistics of the numerical columns
df.describe()

Visualize the distribution of the numerical columns in the dataframe using the `hist()` method of the dataframe. The `hist()` method creates a histogram for each numerical column in the dataframe. A histogram shows the frequency of values in a given range. The x-axis represents the value range and the y-axis represents the frequency of values in that range.

The histograms show that the features have very different scales. We will need to scale the features before training the model to avoid biasing the model towards features with larger values. We also see that some features (e.g. total_rooms, total_bedrooms, population, and households) are skewed due to the presence of outliers. We will transform the features using logarithmic scaling to reduce the skewness. We will perform these steps in the preprocessing section.

In [ ]:
# Plot the distribution for all numerical columns
# bins=50 means to divide the range of values into 50 equal-sized bins
# figsize=(20, 15) means to set the figure size to width=20 and height=15
df.hist(bins=50, figsize=(20, 15))
plt.show()

Check for correlation between the features and the target variable using the `corr()` method of the dataframe. The `corr()` method computes pairwise correlation of columns, excluding NA/null values. The correlation coefficient ranges from -1 to 1. When it is close to 1, it means that there is a strong positive correlation; when the coefficient is close to -1, it means that there is a strong negative correlation; when it is close to zero, it means that there is no linear correlation.

Looking at the last row, we can see that the `median_house_value` has a strong positive correlation with the `median_income` (0.69). We can also see that the `median_house_value` has a weak correlation with the `longitude` (-0.05), `total_bedrooms` (0.05), `population` (-0.02), and `households` (0.07). We will consider removing the weakly correlated features in the preprocessing section.

In [ ]:
# Create a figure
plt.figure(figsize=(12, 8))

# Compute the correlation matrix using the `.corr()` method
correlation_matrix = df.corr()

# Create a mask to block the upper triangle of the correlation matrix
# as it is a mirror image of the lower triangle
# use `np.triu()` to create an upper triangle matrix of True and False
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))

# Plot the heatmap using seaborn's `heatmap()` function
# set `annot=True` to show the correlation values on the heatmap
# set `fmt='.2f'` to round the correlation values to 2 decimal places
# set `mask=mask` to block the upper triangle of the correlation matrix
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', mask=mask)

# show the plot
plt.show()


The correlation matrix can also be viewed as a dataframe to see the correlation coefficients of all the features with each other. Execute the cell below to view the correlation matrix as a dataframe.

In [ ]:
correlation_matrix

We can extract the correlation coefficients of the features with the target variable by selecting the column named `median_house_value` from the correlation matrix dataframe. We have sorted that column in descending order to see the features with the highest correlation with the target variable.

In [ ]:
house_value_corr = correlation_matrix['median_house_value'].sort_values(ascending=False)
print(house_value_corr)

View the correlation between the numerical columns in a Pandas DataFrame using scatter plots. This allows us to visualize linear and non-linear relationships between the features and target if any exists. The plots are generated using the `pairplot()` method of seaborn library. We are using it just to plot the numerical columns against the `median_house_value` column. You can also use the `pairplot()` method to plot all the columns in the dataframe by removing the `x_vars` and `y_vars` parameters from the method. `plot_kws` is a dictionary of keyword arguments that are passed to the scatter plot function inside `pairplot()` method. We are using it to set `alpha` to 0.5 to make the points more transparent.

We can see that the scatter plot for `median_income` and `median_house_value` show a strong positive correlation where the points are have a linear relationship. While the scatter plots for `housing_median_age`, `longitude`, and `latitude`,  show a weak correlation where the points are spread across the plot.

In [ ]:
# Plot the scatter plot between column pairs
sns.pairplot(df, x_vars=house_value_corr.index, y_vars=['median_house_value'], height=5, plot_kws={'alpha': 0.5})

Check the correlation between the location of the houses and the median house value via longitude and latitude. We will use the `plotly.express` library to plot the geographical data. The `scatter_mapbox()` function creates a scatter plot on a map. The `px.scatter_mapbox()` function takes the dataframe as the first argument and the column names of the latitude and lonitude columns as the second and third arguments. The `color` parameter is used to specify the column to be used for coloring the points, which is the median house value in this case. The `hover_data` parameter is used to specify the column data to show when the mouse hover over a pont. The `mapbox_style` parameter is used to specify the style of the map.

This plot is interactive. You can zoom in and out of the map, pan around, and hover over the points to see the data values.

We can see that there are high density of expensive houses in the Bay Area and Los Angeles area, and also a relatively dense expensive houses in the San Diego area. We can see that as we move away from the coast, the house prices decrease. So, we can use the `ocean_proximity` column as a feature to predict the house prices. Additionally, we can create a new feature by using the `longitude` and `latitude` columns to compute the distance from the closest dense areas of expensive houses. This can be done by simply picking a reference point (anchor) from each of the dense expensive area, and computing a given point's distance from the closest anchor. We will perform these steps in the preprocessing section.

In [ ]:
# Use plotly express to plot the scatter plot on a map
fig = px.scatter_mapbox(df, lat="latitude", lon="longitude", color="median_house_value",
                        width=800, height=600,
                        hover_data=df.columns,
                        mapbox_style="open-street-map",
                        zoom=5,
                        )
fig.show()

# Data Preprocessing

Data splitting. We are given one CSV file containing all the data. We will split the data into training, validation, and test sets.
- The training dataset is used to train the machine learning model. The model learns the patterns and relationships within the data from this set. The model adjusts its parameters to minimize the difference between its predictions and the ground truth values provided with this dataset.
- The validation dataset is used to assess the model's performance during training and guide the adjustment of the hyperparameters. Hyperparameters are training/model configurations that the programmer can manually adjust. Since the validation dataset is an independent dataset not used in directly tuning the model's internal parameters, we can use it to assess whether a model is under-fitting or over-fitting.
- The test dataset is kept separate from both the training and validation sets. It is used to assess the final performance of the trained model on unseen data. The test set provides an unbiased evaluation of the model's generalization to new, previously unseen examples.

Building pipelines to preprocess the data. The preprocessing pipeline provides a systematic and efficient way to streamline and automate the data preprocessing steps. It ensures consistent application of preprocessing steps to training, validation, and test data, enhancing model reproducibility, readability, and ease of deployment. In this section, we will build a preprocessing pipeline to perform the following steps:
 - Fill nan values with nearest neighbors
 - Scale the data using logarithmic transformation to reduce the skewness of the data
 - Standardize features by removing the mean and scaling to unit variance using StandardScaler
 - Encode string categorical features using OneHotEncoder
 - Compute new feature based on latitude and longitude (distance from dense area)
 - Drop low correlation features


Reload the data into a Pandas DataFrame to reset the changes made to the dataframe in the previous section. Additionally, we will declare variables to store the names of the numerical, categorical, and label (target) columns.

In [ ]:
# TODO: reload the dataset
csv_path =
df =

# TODO: declare the categorical, numerical, and target column
cat_features =
num_features =
all_features = cat_features + num_features

label =

Split the data into training, validation, and test sets. The training set will be used to train the model, the validation set will be used to evaluate the model during the training process, and the test set will be used to test the final model.

We will be using Scikit-Learn's `train_test_split` function to split the data into training, validation, and test sets. The `train_test_split` function takes in the dataframe as argument and returns the training set and test set. We will further split the training set into training and validation sets. The final split is 60% training, 20% validation, and 20% test.

In [ ]:
# TODO: use a random seed so that we can reproduce the results
# this is important when you want to compare different models in different executions
random_seed =

# TODO: split the data into training and test sets. 80/20 split.
train_set, test_set =

# TODO: split the data into training and validation sets.
# 75/25 split of the training set, which is 60/20 of the original set.
train_set, valid_set =

Separate the feature columns from the label column

In [ ]:
# TODO: separate the features and labels in the training, validation, and test sets
train_X, train_y =
valid_X, valid_y =
test_X, test_y =

Check the feature distribution in the training, validation, and test sets. We want to ensure that the distribution is similar in all the sets.

In [ ]:
# Check the distribution of the training set, validation set, and test set
# For each feature, overlay the histograms of the three sets on the same subplot

# TODO: create a figure with 3 rows and 3 columns
fig, axes =

# TODO: for each feature, plot the histograms of the three sets on the same subplot
for ax, col in zip(, ):

    ax.set_title(col)

# TODO: add a legend to the figure anchor to the top right corner
handles, labels = ax.get_legend_handles_labels()
fig.legend(handles, labels, bbox_to_anchor=)

# TODO: use `plt.tight_layout()` to adjust the padding between and around subplots.

plt.show()


Check the label distribution in the training, validation, and test sets.

In [ ]:
# TODO: check the label distribution of the training set, validation set, and test set
# overlay the histograms of the three sets

plt.figure(figsize=(10, 5))

plt.title(label)
plt.legend()
plt.show()


## Create Preprocessing Pipeline

 - Fill nan values with nearest neighbors
 - Scale the data using logarithmic transformation to reduce the skewness of the data
 - Standardize features by removing the mean and scaling to unit variance using StandardScaler
 - Encode string categorical features using OneHotEncoder
 - Compute new feature based on latitude and longitude (distance from dense area)
 - Drop low correlation features

Below we will create some custom data transformers, which will be used in the preprocessing pipeline. These custom transformers inherits `BaseEstimator` and `TransformerMixin`. These classes are provided by scikit-learn to help in creating custom transformers and estimators that seamlessly integrate with scikit-learn's pipeline.

`BaseEstimator` is the base class for all scikit-learn estimators. An estimator is an object that fits a model based on some training data and is capable of inferring some properties on new data. It can be, for instance, a classifier or a regressor. All estimators implement the fit method (`fit(X, y)`).

`TransformerMixin` is a mixin class for transformers in scikit-learn. A transformer implements the `fit_transform` methods. It provides a default implementation for the `fit_transform` method, which combines fitting and transforming in a single step.

Custom feature engineering class, `MinDistanceFromAnchors`, finds the nearest anchor (based on latitude and longitude information)
        for each given data point and output their Euclidean distance. This transformer is designed to be used within a scikit-learn preprocessing pipeline.

In [ ]:
# Feature Engineering to create new features based on latitude and longitude
class MinDistanceFromAnchors(BaseEstimator, TransformerMixin):
    def __init__(self) -> None:
        '''Finds the nearest anchor (based on latitude and longitude information)
        for each given data point and output their Euclidean distance
        '''
        self.anchors = [(37.38, -122.21), # Bay Area
                        (33.99, -118.5), # Los Angeles
                        (32.82, -117.31), # San Diego
                        ]
        self.num_anchors = len(self.anchors)

    def fit(self, X: np.ndarray, y: np.ndarray=None) -> 'MinDistanceFromAnchors':
        return self

    def transform(self, X: np.ndarray) -> np.ndarray:
        '''Calculate the euclidean distance between each data point and each anchor,
        store the distances in the distances matrix, and filter to nearest anchor distance only.
        '''

        # initialize the distances matrix
        distances = np.zeros((X.shape[0], self.num_anchors))

        # calculate the euclidean distance between each data point and each anchor
        for i in range(self.num_anchors):
            lat_long_points = X.astype(float)
            anchor = np.array(self.anchors[i])
            distances[:, i] = np.sqrt(np.sum((lat_long_points - anchor)**2, axis=1))

        # find the minimum distance out of the distances to the three anchors
        min_distances = np.min(distances, axis=1)

        return min_distances.reshape(-1, 1)


`LogScaler` is a custom transformer that scales the data using logarithmic transformation to reduce the skewness of the data. The `FunctionTransformer` class is used to create a transformer that applies a function to the data. The `FunctionTransformer` class takes in the function to be applied as the first argument.

In [ ]:
# TODO: create a custom transformer using np.log() to log transform the data
log_scaler =

Sets up four preprocessing pipelines. The pipelines consist of several sequential steps, each represented by a tuple with a name and a corresponding transformer.
1. `lat_long_pipeline` is a pipeline for the `latitude` and `longitude` columns. It consists of the `MinDistanceFromAnchors` transformer, which calculates the Euclidean distance from data points to their nearest anchor based on latitude and longitude information. The `StandardScaler` transformer is used to standardize the data by removing the mean and scaling to unit variance.
2. `log_std_pipeline` is a pipeline for the numerical columns with skew distribution. It consists of the `KNNImputer` to fill in any missing values. The `LogScaler` transformer, which scales the data using logarithmic transformation to reduce the skewness of the data. The `StandardScaler` transformer is used to standardize the data by removing the mean and scaling to unit variance.
3. `std_pipeline` is a pipeline for the remaining numerical columns. It consists of the `KNNImputer` to fill in any missing values. The `StandardScaler` transformer is used to standardize the data by removing the mean and scaling to unit variance.
4. `cat_pipeline` is a pipeline for the categorical columns. It consists of the `OneHotEncoder` transformer, which encodes categorical features as a one-hot numeric array. `sparse=False` is used to return a dense array representation in numpy.

In [ ]:
# Latitude and longitude preprocess pipeline
lat_long_pipeline = Pipeline([
    ('min_distance_from_anchors', MinDistanceFromAnchors()),
    ('std_scaler', StandardScaler()),
])

# Create the preprocess pipeline for log scaling and standard scaling features
log_std_pipeline = Pipeline([
        ('imputer', KNNImputer(n_neighbors=10, weights='distance')),
        ('log_scaler', log_scaler),
        ('std_scaler', StandardScaler()),
    ])

# Create the preprocess pipeline for standard scaling features
std_pipeline = Pipeline([
        ('imputer', KNNImputer(n_neighbors=10, weights='distance')),
        ('std_scaler', StandardScaler()),
    ])

# Create the preprocess pipeline for categorical features
cat_pipeline = Pipeline([
        ('one_hot_encoding', OneHotEncoder(sparse=False, categories=[['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR OCEAN', 'NEAR BAY']])),
    ])

We will test out the pipelines on the training set. We will use the `fit_transform` method to fit the pipeline to the training set and transform the training set.

In [ ]:
# make a copy of the training set to test the preprocess pipelines
train_X_copy = train_X.copy()

In [ ]:
# test out the lat_long_pipeline
lat_log_X = train_X_copy[['latitude', 'longitude']]
distances = lat_long_pipeline.fit_transform(lat_log_X)

# add the new feature to the training set
train_X_copy['distance_from_anchor'] = distances

# using plotly express to plot the scatter plot on a map using the new feature as the color
fig = px.scatter_mapbox(train_X_copy, lat="latitude", lon="longitude", color="distance_from_anchor",
                        width=800, height=600,
                        hover_data=train_X_copy.columns,
                        mapbox_style="open-street-map",
                        zoom=5,
                        )
fig.show()

In [ ]:
# testing out the log_std_pipeline
log_std_X = train_X_copy[['total_rooms']]
log_std_X_transformed = log_std_pipeline.fit_transform(log_std_X)

# add the new features to the training set
train_X_copy['log_total_rooms'] = log_std_X_transformed[:, 0]

# plot the histograms of the new features
train_X_copy[['total_rooms']].hist(figsize=(20,5), bins=50, layout=(1,4))
plt.show()
train_X_copy[['log_total_rooms']].hist(figsize=(20,5), bins=50, layout=(1,4))
plt.show()

In [ ]:
# testing out the std_pipeline
log_std_X = train_X_copy[['housing_median_age', 'median_income']]
log_std_X_transformed = std_pipeline.fit_transform(log_std_X)

# add the new features to the training set
train_X_copy['std_housing_median_age'] = log_std_X_transformed[:, 0]
train_X_copy['std_median_income'] = log_std_X_transformed[:, 1]

# plot the histograms of the new features
train_X_copy[['housing_median_age', 'median_income']].hist(figsize=(20,5), bins=50, layout=(1,2))
plt.show()
train_X_copy[['std_housing_median_age', 'std_median_income']].hist(figsize=(20,5), bins=50, layout=(1,2))
plt.show()

In [ ]:
# test out the cat_pipeline
cat_X = train_X_copy[['ocean_proximity']]
cat_X_transformed = cat_pipeline.fit_transform(cat_X)

# print out the first five rows with the categories
print('Value before transformation:')
print(cat_X[:5], end='\n==================\n')

print('Value after transformation:')
print(cat_pipeline['one_hot_encoding'].categories_)
print(cat_X_transformed[:5])

Combine all the pipelines using `ColumnTransformer`. `ColumnTransformer` allows us to apply different preprocessing steps to different columns in the dataset. It is particularly useful when we have a dataset with data types or when we want to apply specific transformations to specific subsets of features.

`ColumnTransformer` takes in a list of tuples, where each tuple contains a name, a transformer, and a list of columns to which the transformer should be applied. The `remainder` parameter is set to `'passthrough'` to ensure that the columns not specified in the list of tuples are passed through without any transformations. Or, we can set `remainder` to `'drop'` to drop the columns not specified in the list of tuples.

In [ ]:
# TODO: create a pipeline with the ColumnTransformer
preprocess_pipeline = ColumnTransformer([




    ], remainder='drop')

Use the full pipeline to fit on the training data, then transform the training and validation data.

In [ ]:
# TODO: fit the pipeline to the training set and transform it
train_X =

# TODO: transform the validation set
valid_X =


In [ ]:
# quick check on the features of the training set
# should contains features `distance from anchor`, `total_rooms`,
# `housing_median_age`, `median_income`, and five features from one-hot `ocean_proximity`.
pd.DataFrame(train_X).hist(bins=50, figsize=(20, 15))
plt.show()

# Train and Evaluate Models

This perhaps is the easier part of the workflow in terms of coding. Scikit-learn provides a wide range of machine learning tools that we can use to train and evaluate on our data. Their APIs are very consistent. We can use the same code to train and evaluate different models by simply changing the model class.

Before we start, we will implement an evaluation function. This function `predict_and_print_metrics` takes a regression model, input features (X), target values (y), and a dataset name. It then predicts the target values using the model, calculates and prints the Mean Absolute Error (MAE) and the Root Mean Squared Error (RMSE).

MAE (Mean Absolute Error):
- MAE is calculated as the mean of absolute differences between predicted and true values.
- It treats all errors equally, regardless of their magnitude.
- $MAE = \frac{1}{N}\sum_{i=1}^{N} |y_i - \hat{y_i}|$ where $y_i$ is true value of sample $i$, $\hat{y_i}$ is the model prediction value of sample $i$, ad $N$ is the number of samples

RMSE (Root Mean Squared Error):
- RMSE is calculated as the square root of the mean of squared differences between predicted and true values.
- It penalizes large errors more heavily than small ones.
- $RMSE = \sqrt{\frac{1}{N}\sum_{i=1}^{N} (y_i - \hat{y_i})^2}$ where $y_i$ is true value of sample $i$, $\hat{y_i}$ is the model prediction value of sample $i$, ad $N$ is the number of samples





In [ ]:
# TODO: import the mean squared error and mean absolute error metrics


# TODO: create a function to predict and print the metrics
def predict_and_print_metrics(model, X: np.ndarray, y: np.ndarray, name: str) -> None:
    # predict the target values
    y_pred =

    # compute and print the metrics
    print(f'{name} MAE: {}')
    print(f'{name} RMSE: {}')


Train a Linear Regression model. The training can be done in three lines of code. First, we import the model class from the Scikit-Learn library. Then, we create an instance of the model class. Third, we call the `fit` method of the model instance to train the model. The `fit` method takes in the training features and labels as arguments. The model learns the patterns and relationships within the data. Then, we use the trained model to predict the target values for the training and validation set, and we evaluate the model using the evaluation metrics mentioned above.

In [ ]:
# TODO: import the linear regression model


# TODO: instantiate and train the model
lin_reg =

# TODO: evaluate the model on the training set and validation set

We will train and evaluate another model (K-neighbors regressor). We will use the same code to train and evaluate different models by simply changing the model class.

In [ ]:
# TODO: import the K-neighbors regressor


# TODO: instantiate and train the model
neigh =

# TODO: evaluate the model on the training set and validation set

In the case where the model has hyperparameters, we can either write our own code to search for the best hyperparameters or use Scikit-Learn's `GridSearchCV` class to search for the best hyperparameters. We start off by writing our own code to search for the best hyperparameters.

In [ ]:
# Train a k-neighbors regressor with search for the best hyperparameters

# TODO: create lists to store the train and validation scores
train_scores =
valid_scores =

# TODO: create a list of hyperparamters to try out
neighbors =

# TODO: loop through the parameters and fit the model
# evaluate the model on the train set and validation set
for neighbor in neighbors:
    neigh =

    # evaluate the model on the train set
    train_y_pred =
    train_mae =
    train_scores.append(train_mae)

    # evaluate the model on the validation set
    valid_y_pred =
    valid_mae =
    valid_scores.append(valid_mae)

# TODO: plot the learning curves
plt.plot(, , label=)
plt.plot(, , label=)
plt.xlabel('number of neighbors')
plt.ylabel('MAE')
plt.legend()
plt.show()

# TODO: print the neighbors and the corresponding scores
for neighbor, train_score, valid_score in zip(, , ):
    print(f'neighbors: {neighbor}, train score: {train_score}, valid score: {valid_score}')

Use `GridSearchCV` to search for the best hyperparameters. `GridSearchCV` takes in the model class, hyperparameter grid, evaluation metric (scorer), and number of folds as arguments. It then searches for the best hyperparameters based on the evaluation metric. We will use the same evaluation metric as before, MAE. We will combine the training and evaluation data into one dataset. This is because `GridSearchCV` uses cross-validation to evaluate the model. We will use 5-fold cross-validation, which means that the dataset will be split into 5 folds. The model will be trained and evaluated 5 times, each time using a different fold as the evaluation set. The final evaluation metric will be the average of the 5 evaluations.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

# TODO: combine the train and validation sets for GridSearchCV
train_valid_X =
train_valid_y =

# TODO: Specify the parameter grid for the grid search
param_grid = {

}

# TODO: Define your MAE evaluation metric function
def custom_metric(y_true, y_pred):
    # Implement your custom metric calculation here
    # This example uses the mean absolute error (MAE) as a placeholder
    mae = mean_absolute_error(y_true, y_pred)
    return mae

# TODO: Make the metric a scorer using make_scorer
scorer = make_scorer(, greater_is_better=)


# TODO: Create the KNeighborsRegressor
neigh =

# TODO: Create the GridSearchCV object with custom scoring
grid_search = GridSearchCV(
    ,
    ,
    scoring=,
    cv=5  # You can specify the number of folds for cross-validation
)

# TODO: Fit the grid search to your data


# Access the best model and its parameters
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

# Print the results
print(f"Best Model: {best_model}\n")
print(f"Best Parameters: {best_params}\n")

# You can also access other grid search results
best_index = grid_search.best_index_
print('All mean test scores: \n', grid_search.cv_results_['mean_test_score'], '\n')
print('Best hyperparamter mean test scores: \n', grid_search.cv_results_['mean_test_score'][best_index])


# Putting it all together in a pipeline and export pipeline

We will now combine the preprocessing pipeline and the best model into one pipeline.

In [ ]:
# Combine the preprocessing pipeline and the best model into a new pipeline
pipeline = Pipeline([
    ('preprocess', preprocess_pipeline),
    ('model', best_model)
])

Save the pipeline using `joblib.dump` and load it back for predicting the test set

In [ ]:
# Save the pipeline
joblib.dump(pipeline, '../pipelines/house_price_pipeline.joblib')

In [ ]:
# Load the pipeline
pipeline = joblib.load('../pipelines/house_price_pipeline.joblib')

# Evaluate the pipeline on the test set
predict_and_print_metrics(pipeline, test_X, test_y, 'Test')


Check to see how the model performs on the test set. We will see how much percentage the predictions deviate from the actual values.

In [ ]:
percentage_diff = (pipeline.predict(test_X) - test_y)/test_y * 100

plt.figure(figsize=(12,8))
plt.subplot(2,1,1)
plt.hist(percentage_diff, bins=100)
plt.xlabel('Percentage Difference')
plt.ylabel('Count')
plt.xlabel('Percentage Difference')
plt.xlim(-100,400)
plt.xticks(np.arange(-100, 400, 20))

plt.subplot(2,1,2)
sns.boxplot(percentage_diff, orient='h')
plt.xlim(-100,400)
plt.xlabel('Percentage Difference')
plt.show()